## Red-Black Tree

Type of balance search tree
1. node is either red or black
2. the root and leaf (nil) are black
3. if a node is red, then its child is black
4. all paths from a node to its NIL descendants contain the same number of black nodes

search
insert
remove

remove require rotations.


In [236]:
class Node:
    def __init__(self, value, left=None, right=None, parent=None, color="RED"):
        self.value = value
        self.parent = parent
        self.left = left
        self.right = right
        self.color = color


    def print_node(self):
        if self.color == "RED":
            color_code = "\033[91m"  # Red color escape sequence
        else:
            color_code = "\033[0m"   # Reset color escape sequence

        print("\n\nValue:", self.value)
        print("Parent:", self.parent.value if self.parent else None)
        print("Left Child:", self.left.value if self.left else None)
        print("Right Child:", self.right.value if self.right else None)
        print("Color:", f"{color_code}{self.color}\033[0m")

In [237]:
class RedBlackTree:
    def __init__(self):
        self.nil = Node(value=None, color="BLACK")
        self.root = self.nil

    def inOrderTraversal(self,node):
        if node:
            print(f"{node.value} - { node.color}")
            #node.print_node()
            self.inOrderTraversal(node.left)
            self.inOrderTraversal(node.right)

    def search(self, value):
        current = self.root
        while current != self.nil:
            if value == current.value:
                print(f"Found Node with Val {value}")
                return current  
            elif value < current.value:
                current = current.left
            else:
                current = current.right
        print(f"Couldn't find Node with value: {value}")
        return None

    def insert(self, value):
        # Create node
        new_node = Node(value=value, left=self.nil, right=self.nil, parent=self.nil)
        parent = None

        # Case insert at root
        if self.root == self.nil:
            self.root = new_node
        else:
            # traverse to find insertion location
            current = self.root
            print("ROOT: _------------___---")
            #self.root.print_node()
            print(current.value)
            while current != self.nil:
                parent = current
                current.print_node()
                if new_node.value < current.value:
                    print(" - ")
                    current = current.left
                else:
                    print(" - ")
                    current = current.right

            # Assignment
            new_node.parent = parent
            if new_node.value < parent.value:
                parent.left = new_node
            else:
                parent.right = new_node

            new_node.left = self.nil
            new_node.right = self.nil
            new_node.color = "RED"

        print("Finish Insertion")
        self.recolor(new_node)

    def getRoot(self):
        return self.root
    
    def recolor(self, node):
        while node.parent.color == "RED":  # Adjusted condition
            if node.parent == node.parent.parent.left:
                uncle = node.parent.parent.right
                if uncle.color == "RED":
                    node.parent.color = "BLACK"
                    uncle.color = "BLACK"
                    node.parent.parent.color = "RED"
                    node = node.parent.parent
                else:
                    if node == node.parent.right:
                        node = node.parent
                        self.lRotation(node)
                    node.parent.color = "BLACK"
                    node.parent.parent.color = "RED"
                    self.rRotation(node.parent.parent)
            else:
                uncle = node.parent.parent.left
                if uncle.color == "RED":
                    node.parent.color = "BLACK"
                    uncle.color = "BLACK"
                    node.parent.parent.color = "RED"
                    node = node.parent.parent
                else:
                    if node == node.parent.left:
                        node = node.parent
                        self.rRotation(node)
                    node.parent.color = "BLACK"
                    node.parent.parent.color = "RED"
                    self.lRotation(node.parent.parent)

        self.root.color = "BLACK"  # Set root to black

    def rRotation(self,node):
        y = node.left
        node.left = y.right

        #check if node.left.right enodeists
        if node.left != self.nil:
            node.left.parent = node

        y.parent = node.parent
        #check if node is root node
        if node.parent == self.nil:
            self.root = y
        elif node == node.parent.right:
            node.parent.right = y
        else:
            node.parent.left = y
        node.parent = y
        y.right = node
        #print("-----Right Rotation")

    def lRotation(self,node):
        y = node.right
        node.right = y.left

        #check if node.left.right enodeists
        if node.right != self.nil:
            node.right.parent = node

        y.parent = node.parent
        #check if node is root node
        if node.parent == self.nil:
            self.root = y
        elif node == node.parent.left:
            node.parent.left = y
        else:
            node.parent.right = y
        node.parent = y
        y.left = node
        #print("-----Left Rotation")

    def transplant(self,node,child):
        # If node is root
        if node.parent == self.nil:
            self.root = child

        # Determines whether node is right or left child
        elif node == node.parent.left:
            node.parent.left = child
        else:
            node.parent.right = child

    def remove(self,node):
        isNode = node
        if isNode:
            print("Removing")
            if node == self.root:
                self.root = self.nil
            # Case if node is leaf
            if isNode.left == self.nil and isNode.right == self.nil:
                if isNode.value > isNode.parent.value:
                    isNode.parent.right = self.nil
                else:
                    isNode.parent.left = self.nil
                return
            
            # Case if node contains 1 child
            if isNode.left == self.nil or isNode.right == self.nil:
               child = isNode.left if isNode.left != self.nil else isNode.right
               self.transplant(isNode, child)
               if node.color == "BLACK":
                    if child.color == "RED":
                        child.color = "BLACK"
                    else:
                        self.finodeDoubleBlack(child)
            # Case if node contains 2 children
            else:
                successor = self.findSuccessor(node.right)
                node.value = successor.value
                self.remove(successor)
    def findSuccessor(self, node):
        current = node
        while current.left != self.nil:
            current = current.left
        return current


    def finodeDoubleBlack(self, current):
            while current != self.root and current.color == "BLACK":
                if current == current.parent.left:
                    uncle = current.parent.right
                    if uncle.is_red():
                        uncle.color = "BLACK"
                        current.parent.color = "RED"
                        self.left_rotate(current.parent)
                        uncle = current.parent.right
                    if uncle.left.color == "BLACK" and uncle.right.color == "BLACK":
                        uncle.color = "RED"
                        current = current.parent
                    else:
                        if uncle.right.color == "BLACK":
                            uncle.left.color = "BLACK"
                            uncle.color = "RED"
                            self.right_rotate(uncle)
                            uncle = current.parent.right
                        uncle.color = current.parent.color
                        current.parent.color = "BLACK"
                        uncle.right.color = "BLACK"
                        self.left_rotate(current.parent)
                        current = self.root
                else:
                    uncle = current.parent.left
                    if uncle.is_red():
                        uncle.color = "BLACK"
                        current.parent.color = "RED"
                        self.right_rotate(current.parent)
                        uncle = current.parent.left
                    if uncle.right.color == "BLACK" and uncle.left.color == "BLACK":
                        uncle.color = "RED"
                        current = current.parent
                    else:
                        if uncle.left.color == "BLACK":
                            uncle.right.color = "BLACK"
                            uncle.color = "RED"
                            self.left_rotate(uncle)
                            uncle = current.parent.left
                        uncle.color = current.parent.color
                        current.parent.color = "BLACK"
                        uncle.left.color = "BLACK"
                        self.right_rotate(current.parent)
                        current = self.root
            current.color = "BLACK"

    def printTree(self):
        self.p(self.root)

    def p(self, node, parent_color="WHITE"):
        if node != self.nil:
            left_child_color = "RED" if node.left.color == "RED" else parent_color
            right_child_color = "RED" if node.right.color == "RED" else parent_color
            left_child_value = node.left.value if node.left != self.nil else "nil"
            right_child_value = node.right.value if node.right != self.nil else "nil"
            
            if node.color == "RED":
                color_code = "\033[91m"  # Red color escape sequence
            else:
                color_code = "\033[0m"
            
            print(f"{color_code}{node.value} (", end="")
            self.p(node.left, left_child_color)
            print(", ", end="")
            self.p(node.right, right_child_color)
            print(f")\033[0m", end=" ")


   

In [238]:
tree = RedBlackTree()
tree.insert(10)
tree.insert(20)
print("-----------------------")
tree.insert(30)
tree.insert(15)
tree.insert(25)
tree.insert(5)

print("\nInorder Traversal of Red-Black Tree:")
tree.getRoot()

tree.printTree()


Finish Insertion
ROOT: _------------___---
10


Value: 10
Parent: None
Left Child: None
Right Child: None
Color: BLACK
 - 
Finish Insertion
-----------------------
ROOT: _------------___---
10


Value: 10
Parent: None
Left Child: None
Right Child: 20
Color: BLACK
 - 


Value: 20
Parent: 10
Left Child: None
Right Child: None
Color: RED
 - 
Finish Insertion
ROOT: _------------___---
20


Value: 20
Parent: None
Left Child: 10
Right Child: 30
Color: BLACK
 - 


Value: 10
Parent: 20
Left Child: None
Right Child: None
Color: RED
 - 
Finish Insertion
ROOT: _------------___---
20


Value: 20
Parent: None
Left Child: 10
Right Child: 30
Color: BLACK
 - 


Value: 30
Parent: 20
Left Child: None
Right Child: None
Color: BLACK
 - 
Finish Insertion
ROOT: _------------___---
20


Value: 20
Parent: None
Left Child: 10
Right Child: 30
Color: BLACK
 - 


Value: 10
Parent: 20
Left Child: None
Right Child: 15
Color: BLACK
 - 
Finish Insertion

Inorder Traversal of Red-Black Tree:
20 (10 (5 (, ) , 15 (, ) )

In [239]:
rb_tree = RedBlackTree()

# Insert some values into the tree
values = [10, 5, 15, 3, 7, 12, 18,88,66,72,4,11,1,23]
for value in values:
    rb_tree.insert(value)

# Print the tree
print("Red-Black Tree:")
rb_tree.inOrderTraversal(rb_tree.root)

Finish Insertion
ROOT: _------------___---
10


Value: 10
Parent: None
Left Child: None
Right Child: None
Color: BLACK
 - 
Finish Insertion
ROOT: _------------___---
10


Value: 10
Parent: None
Left Child: 5
Right Child: None
Color: BLACK
 - 
Finish Insertion
ROOT: _------------___---
10


Value: 10
Parent: None
Left Child: 5
Right Child: 15
Color: BLACK
 - 


Value: 5
Parent: 10
Left Child: None
Right Child: None
Color: RED
 - 
Finish Insertion
ROOT: _------------___---
10


Value: 10
Parent: None
Left Child: 5
Right Child: 15
Color: BLACK
 - 


Value: 5
Parent: 10
Left Child: 3
Right Child: None
Color: BLACK
 - 
Finish Insertion
ROOT: _------------___---
10


Value: 10
Parent: None
Left Child: 5
Right Child: 15
Color: BLACK
 - 


Value: 15
Parent: 10
Left Child: None
Right Child: None
Color: BLACK
 - 
Finish Insertion
ROOT: _------------___---
10


Value: 10
Parent: None
Left Child: 5
Right Child: 15
Color: BLACK
 - 


Value: 15
Parent: 10
Left Child: 12
Right Child: None
Color: BLAC

In [240]:
target = rb_tree.search(10)
rb_tree.printTree()
print("")
rb_tree.remove(target)

rb_tree.printTree()

print("")
print("Searching")
rb_tree.search(1)

Found Node with Val 10
15 (10 (5 (3 (1 (, ) , 4 (, ) ) , 7 (, ) ) , 12 (11 (, ) , ) ) , 66 (18 (, 23 (, ) ) , 88 (72 (, ) , ) ) ) 
Removing
Removing
15 (11 (5 (3 (1 (, ) , 4 (, ) ) , 7 (, ) ) , 12 (, ) ) , 66 (18 (, 23 (, ) ) , 88 (72 (, ) , ) ) ) 
Searching
Found Node with Val 1
